Preprocessing planet imagery for median mosaic

1. mask clouds
2. assign nodata for partial overlap scenes within bounds of supercell
3. median for each band and composite

In [ ]:
from shapely.geometry import Polygon
import pyproj
import numpy as np
import geopandas as gpd
import rasterio as rio
from rasterio import mask
import rasterstats as rstats
import skimage as ski
import matplotlib.pyplot as plt
import glob
import os
from rasterio.plot import reshape_as_raster, reshape_as_image
import json
os.chdir('../')
from filter_callable import cloud_shadow_stats
sr_pattern = "/home/rave/cloud-free-planet/notebooks/jan_april_2018_100ovp_50maxcloud/*SR*.tif"
img_paths = glob.glob(sr_pattern)
udm_pattern = "/home/rave/cloud-free-planet/notebooks/jan_april_2018_100ovp_50maxcloud/*udm*.tif"
udm_paths = glob.glob(udm_pattern)

img_paths = sorted(img_paths)

udm_paths = sorted(udm_paths)

def filter_date(lst, month_int, month_digit1, month_digit2):
    """Takes a list of udms or images and returns filtered by month lists
    example
    imgs_may = filter_date(img_paths, '05', 12, 14)
    udms_may = filter_date(udm_paths, '05', 12, 14)"""
    return [i for i in lst if os.path.basename(i)[month_digit1:month_digit2] == month_int]


def test_udm_mask(image_path, udm_path):
    """
    Masks a planet image by it's udm
    """

    img = rio.open(image_path)
    img_meta = img.profile
    img_array = np.array([img.read(1), img.read(2), img.read(3), img.read(4)])
    mask = rio.open(udm_path).read(1)[..., :] == 0 # 0 is the value in the udm that corresponds to good data
    masked = np.where(mask,img_array, 0)
    return np.swapaxes(np.swapaxes(masked, 0, 2),0, 1)


def test_custom_mask(image_path):
    """
    Masks a custom cloud mask from filter_callable
    """

    img = rio.open(image_path)
    img_meta = img.profile
    cloud_arr, shadow_arr = cloud_shadow_stats(image_path)
    cloud_arr = np.logical_not(cloud_arr.astype(bool))
    shadow_arr = np.logical_not(shadow_arr.astype(bool))
    masked = np.where(cloud_arr,img.read(), False)
    masked = np.where(shadow_arr,masked, False)
    return np.swapaxes(np.swapaxes(masked, 0, 2),0, 1)

from skimage import exposure

def percentile_rescale(arr, plow=1, phigh=99):
    '''
    Rescales and applies other exposure functions to improve image vis. 
    http://scikit-image.org/docs/dev/api/skimage.exposure.html#skimage.exposure.rescale_intensity
    '''
    rescaled_arr = np.zeros_like(arr)
    for i in range(0,arr.shape[-1]):
        val_range = (np.percentile(arr[:,:,i], plow), np.percentile(arr[:,:,i], phigh))
        rescaled_channel = exposure.rescale_intensity(arr[:,:,i], val_range)
        rescaled_arr[:,:,i] = rescaled_channel
#     rescaled_arr= exposure.adjust_gamma(rescaled_arr, gamma=1) #adjust from 1 either way
#     rescaled_arr= exposure.adjust_sigmoid(rescaled_arr, cutoff=.50) #adjust from .5 either way 
    return rescaled_arr
def normalize(arr):
    ''' Function to normalize an input array to 0-1 '''
    arr_max = arr.max()
    return arr / arr_max

def reorder_to_rgb(image):
    '''reorders wv2 bands ordered like RGBNRGN for off/onseason
    to blue, red, green for imshow
    '''
    blue = normalize(image[:,:,2])
    green = normalize(image[:,:,1])
    red = normalize(image[:,:,0])
    nir = normalize(image[:,:,3])
    return np.stack([red, green, blue], axis=-1) 

def is_even(x):
    if x%2 == 1:
        return False
    else:
        return True
def is_div_by(x, y):
    if x%y == 0:
        return True
    else:
        return False
def even_only(iter):
    return [x for x in iter if is_even(x)]
def odds_only(iter):
    return [x for x in iter if not is_even(x)]
def by_3_only(iter):
    return [x for x in iter if is_div_by(x, 3)]
def by_3_no_odd_only(iter):
    return [x for x in iter if is_div_by(x, 3) and is_even(x)]
def even_no_3_only(iter):
    return [x for x in iter if not is_div_by(x, 3) and is_even(x)]
def odd_no_3_only(iter):
    return [x for x in iter if not is_div_by(x, 3) and not is_even(x)]

In [ ]:
meta_pattern = "/home/rave/cloud-free-planet/notebooks/jan-may/*metadata.json"
meta_paths = glob.glob(meta_pattern)
meta_paths = sorted(meta_paths)

# Below functions are for ATSA

need to turn the tifs into ENVI standard files like the example and stack the time series. Either stack in envi or stack as tifs before bringing to envi

In [ ]:

img_paths_with_meta = keep_imgs_with_meta(img_paths,meta_paths)

In [ ]:
def get_sun_elevation_azimuth(path):
    with open(path) as f:
        metadata = json.load(f)
    return (metadata['properties']['sun_elevation'], metadata['properties']['sun_azimuth'])

angles = list(map(get_sun_elevation_azimuth, meta_paths))
with open('angles.txt', 'w') as f:
    for tup in angles:
        f.write('%s %s\n' % tup)

def save_blank_water_mask(path):
    """Used for testing ATSA where we know ther eis no water
    In the future we can use MOD44W water mask product or something else
    """
    test = rio.open(path)
    meta = test.profile
    meta.update(count=1) # update since we are writing a single band
    b1_array, b2_array, b3_array, b4_array = test.read()
    fake_mask = np.zeros(b1_array.shape)
    with rio.open('fake_mask.tif', 'w', **meta) as dst:
        dst.write(fake_mask.astype('uint16'), 1)
        
save_blank_water_mask(img_paths_with_meta[0])

with rio.open(img_paths_with_meta[0]) as test:
    print(test.read().shape)

def stack_t_series(paths, stackname):
    """"
    Stack third axis-wise. all 
    tifs must be same extent and in sorted order by date
    """
    arrs = [ski.io.imread(path) for path in paths]
    stacked = reshape_as_raster(np.dstack(arrs))
    img = rio.open(paths[0])
    meta=img.profile
    meta.update(count=len(arrs)*arrs[0].shape[2])
    with rio.open(stackname, 'w',  **meta) as dst:
        dst.write(stacked)
    print("Saved Time Series with " + str(len(arrs)) + " images and " + str(arrs[0].shape[2]) + " bands each")

stack_t_series(img_paths_with_meta, "stacked.tif")

In [ ]:
imgs_april = filter_date(img_paths, '04', 4,6)
udms_april = filter_date(udm_paths, '04', 4,6)
imgs_march = filter_date(img_paths, '03', 4,6)
udms_march = filter_date(udm_paths, '03', 4,6)
imgs_feb = filter_date(img_paths, '02', 4,6)
udms_feb = filter_date(udm_paths, '02', 4,6)
imgs_jan = filter_date(img_paths, '01', 4,6)
udms_jan = filter_date(udm_paths, '01', 4,6)

In [ ]:
sr_pattern = "/home/rave/cloud-free-planet/notebooks/clipped-may/*SR*.tif"
img_paths = glob.glob(sr_pattern)
udm_pattern = "/home/rave/cloud-free-planet/notebooks/clipped-may/udms/*udm.tif"
udm_paths = glob.glob(udm_pattern)

img_paths = sorted(img_paths)

udm_paths = sorted(udm_paths)

imgs_may = filter_date(img_paths, '05', 12,14)
udms_may = filter_date(udm_paths, '05', 12,14)

Call with lyndon on cloud

- date weighting (lyndon)
- weighted mean 1-abs(cloud index-mean(cloud_index)) at each pixel. index could be ndvi, where low values are cloudy or https://www.researchgate.net/publication/327262968_CloudShadow_Detection_Framework_based_on_Spectral_Indices_for_MultiHyperspectral_Optical_Remote_Sensing_Imagery
- shadows
- shadow spatial regression

Cloud Mask Questions

* How can we get rid of the confusion between bright urban surfaces and clouds? Currently we are using the two cloud indices. Either must be satisfied to be classified as a cloud

    ci_one_arr = (b4_array*3)/(b1_array+b2_array+b3_array)
    ci_two_arr = (b4_array+b1_array+b2_array+b3_array)/4

* The mask is very sensitive to the coefficient used to make the second threshold. The second CI threshold is determined like so. Are there better ways to determine this threshold? Basically looking for alternative method suggestions.

    ci_thresh_two = np.mean(ci_two_arr) + ci_coef_two*(np.max(ci_two_arr)-np.mean(ci_two_arr))





In [ ]:
%load_ext autoreload

%autoreload 2
from filter_callable import *
plt.style.use('dark_background')
test_path = imgs_may[12]
test_udm_path = udms_may[12]
plt.figure(figsize=(9,9))
test = rio.open(test_path)
b1_array, b2_array, b3_array, b4_array = test.read()
band_list = [b1_array ,b2_array, b3_array, b4_array]
stacked = np.dstack(band_list)
plt.imshow(percentile_rescale(reorder_to_rgb(stacked)))

plt.figure(figsize=(9,9))
test_cloud = cloud_stats_zhen(test_path, ci_coef_two=1/8, ci_thresh_one=.5, med_kernel=5, object_size_thresh=100)
plt.title("zhen et al. mask")
plt.imshow(test_cloud)

plt.figure(figsize=(9,9))
arr = ski.io.imread(test_udm_path)
plt.title("udm mask")
plt.imshow(arr)

### Bad Shadow Mask Questions
* can we improve this with match filtering using the cloud masks as templates and the sun elevation, azimuth and view angle information? 

* How could we assume or estimate the cloud height without brightness temperature (similar to what Fmask does)? 

* How could we do a match filtering that doesn't depend on scene metadata if we can't assume cloud height?

* Because match filtering wouldn't be able to account for shadows at the edge of the image, we'd probably want to run this on the whole Planet scene, focusing more on

In [ ]:
from filter_callable import *
test = rio.open(imgs_may[0])
b1_array, b2_array, b3_array, b4_array = test.read()
band_list = [b1_array ,b2_array, b3_array, b4_array]
stacked = np.dstack(band_list)

shadow_array_initial = initial_shadow_filter(stacked, shadow_reflectance_thresh=3100)
shadow_array = shadow_size_shape_filter(shadow_array_initial, 
                                        object_size_thresh=200, 
                                        eccentricity_thresh=.95, 
                                        peri_to_area_ratio=.3)
print(np.count_nonzero(shadow_array_initial))
plt.figure(figsize=(9,9))
plt.imshow(percentile_rescale(reorder_to_rgb(stacked)))
plt.figure(figsize=(9,9))
plt.imshow(shadow_array)


In [ ]:
plt.style.use('dark_background')
plt.hist(stacked[:,:,3].flatten(), bins='auto')
plt.show()

The below block plots all the images and masks for a month, needs to be functionalized

In [ ]:
fig=plt.figure(figsize=(20, 350))
columns = 3
rows = 78
plot_indices = list(range(1,78))
path_indices = list(range(1, 26))
for i, even, odd, by_3 in zip(path_indices, even_no_3_only(plot_indices), odd_no_3_only(plot_indices), by_3_only(plot_indices)):
    udm_masked = test_udm_mask(imgs_may[i], udms_may[i])
    custom_masked = test_custom_mask(imgs_may[i])
    udm_masked = normalize(udm_masked)
    custom_masked = normalize(custom_masked)
    udm_masked_rgb = reorder_to_brg(udm_masked)
    custom_masked_rgb = reorder_to_brg(custom_masked)
    fig.add_subplot(rows, columns, even)
    plt.imshow(percentile_rescale(udm_masked_rgb, plow=.5, phigh=99.5))
    plt.title("UDM")
    fig.add_subplot(rows, columns, odd)
    plt.imshow(percentile_rescale(custom_masked_rgb, plow=.5, phigh=99.5))
    plt.title("Custom Mask")
    fig.add_subplot(rows, columns, by_3)
    img = ski.io.imread(imgs_may[i])
    plt.imshow(img[:,:,0])
    plt.title("Blue Band No Mask")                     
plt.tight_layout()
plt.savefig("all_may_images.png", bbox_inches='tight')
plt.show()



In [ ]:
def make_median_composite(imgs, udms):
    """Take slist of image arrays and udm arrays"""
    udm_mask_arrs = [test_udm_mask(img, udm).astype(float) for img,udm in zip(imgs, udms)]
    custom_mask_arrs = [test_custom_mask(img).astype(float) for img in imgs_may]
    for arr in udm_mask_arrs:
        arr[arr==0] = np.nan
    for arr in custom_mask_arrs:
        arr[arr==0] = np.nan
    udm_masked_med = np.nanmedian(udm_mask_arrs, axis=0)
    custom_masked_med = np.nanmedian(custom_mask_arrs, axis=0)
    return (udm_masked_med, custom_masked_med)

In [ ]:
jan_composites = make_median_composite(imgs_jan, udms_jan)
feb_composites = make_median_composite(imgs_feb, udms_feb)
march_composites = make_median_composite(imgs_march, udms_march)
april_composites = make_median_composite(imgs_april, udms_april)

In [ ]:
may_composites = make_median_composite(imgs_may, udms_may)

In [ ]:
imgs_may_april = imgs_may +imgs_april
udms_may_april = udms_may + udms_april

may_april_composites = make_median_composite(imgs_may_april, udms_may_april)

In [ ]:
imgs_may_jan = imgs_may +imgs_april + imgs_march +imgs_feb +imgs_jan
udms_may_jan = udms_may + udms_april + imgs_march +imgs_feb +imgs_jan

may_jan_composites = make_median_composite(imgs_may_jan, udms_may_jan)

In [ ]:
plt.figure(figsize=(10,10))
ski.io.imshow(
    percentile_rescale(
        reorder_to_brg(
            normalize(
                np.nan_to_num(may_jan_composites[0])
            ))))
plt.title("jan to may median composite (udm)")
plt.savefig('udm_composite_may_jan.png')

In [ ]:
plt.figure(figsize=(10,10))
ski.io.imshow(
    percentile_rescale(
        reorder_to_brg(
            normalize(
                np.nan_to_num(custom_masked_med)
            ))))
plt.title("custom may composite")
plt.savefig('custom_composite_may.png')

Below is older code for saving out the masked images when I was using GRASS GIS to do the median compositing. Solved median compositing in numpy now above

In [ ]:

def custom_mask(image_path):
    """
    Masks a custom cloud mask from filter_callable
    """

    img = rio.open(image_path)
    img_meta = img.profile
    cloud_arr, shadow_arr = cloud_shadow_stats(image_path)
    masked = np.where(cloud_arr,img.read(), 0)
    # need to fix this shdow mask #masked = np.where(shadow_arr,masked, 0)
    with rio.open(image_path[0:-12]+'custom_masked.tif', 'w', **img_meta) as dst:
        dst.write(masked) # when mask is true yield img, otherwise yield nan



In [ ]:
list(map(custom_mask, img_paths))

In [ ]:
def check_udm_and_mask(img_path):
    
    def udm_mask(image_path, udm_path):
        """
        Masks a planet image by it's udm
        """
    
        img = rio.open(image_path)
        img_meta = img.profile
        img_array = np.array([img.read(1), img.read(2), img.read(3), img.read(4)])
        mask = rio.open(udm_path).read(1)[..., :] == 0 # 0 is the value in the udm that corresponds to good data
        masked = np.where(mask,img_array, 0)
        with rio.open(img_path[0:-12]+'masked.tif', 'w', **img_meta) as dst:
            dst.write(masked) # when mask is true yield img, otherwise yield nan
        
    if os.path.isfile(img_path) and os.path.isfile(img_path[0:-12]+'_DN_udm_clip.tif'):
        udm_mask(img_path, img_path[0:-12]+'_DN_udm_clip.tif')
    else:
        pass
    
map(check_udm_and_mask, img_paths)

In [ ]:
masked_pattern = "/home/rave/cloud-free-planet/mosaic-tests/*custom*.tif"
masked_paths = glob.glob(masked_pattern)

In [ ]:
import functools
def save_single_bands(t):
    """
    Takes a band index and a path. Saves the
    specified band of each tif and names it by the band index. 
    """
    (band_index, path) = t
    img = rio.open(path)
    meta = img.profile
    meta.update(count=1) # update since we are writing a single band
    arr = img.read(band_index)
    dst_path = os.path.join(os.path.dirname(path),"Band_"+str(band_index)+"_"+os.path.basename(path))
    with rio.open(dst_path, 'w', **meta) as dst:
        dst.write(arr, 1)


In [ ]:
for i in [1, 2, 3, 4]:
    list(map(save_single_bands, [(i, x) for x in masked_paths]))

The above functions take raw analytic_sr that has been clipped and the udm masks and masks the data, then saves each band seperately out. Median compositing is done in GRASS GIS, because couldn't figure out how to median composite partially overlapping arrays wtih rasterio and numpy

# Code for creating Coarse Africa Grid in Decimal Degrees, WGS84 Datum

Below is the coarse layout defined in geopyspark to merge model outputs. Each tile is .0512 degrees by .0512 degrees. need to make a shapely multi line string or polygon bject that can be used to save out tiles as geojsons (only if they intersect an aoi) so that we can use each tile geojson with porder

coarse_layout = gps.LayoutDefinition(gps.Extent(-17.541, -34.845, 51.4766, 37.5518), gps.TileLayout(1348, 1414, 4096, 4096))

In [ ]:
from shapely.geometry import Polygon
import pyproj
import numpy as np
import geopandas as gpd
import rasterio as rio
from rasterio import mask
import rasterstats as rstats
import skimage as ski
import matplotlib.pyplot as plt
import glob
import os
from rasterio.plot import reshape_as_raster, reshape_as_image
import json

def layout_to_geojson(extent=(-17.541, -34.845, 51.459, 37.555), stepsize=.0025, output_name="./supercell_tile_grid.geojson"):
    """
    :param extent: (lonmin, latmin, lonmax, latmax) checked this by 
    looking at coarse layout in run_geopyspark
    
    :param stepsize: width and height of tile in degrees, checked 
    this by measuring extent of probability image tiles
    
    :returns: nothing, saves a geodataframe to a geojson file that encodes the supercell grid
    """
    
    (lonmin, latmin, lonmax, latmax) = extent
    cols = (lonmax - lonmin)/stepsize
    rows = (latmax - latmin)/stepsize
    # Top left corner of grid, where we start to build the gdf
    XleftOrigin = lonmin
    XrightOrigin = lonmin + stepsize
    YtopOrigin = latmax
    YbottomOrigin = latmax - stepsize
    polygons = []
    
    for i in range(int(cols)):
        
        Ytop = YtopOrigin
        Ybottom =YbottomOrigin
        for j in range(int(rows)):
            polygons.append(Polygon([(XleftOrigin, Ytop), (XrightOrigin, Ytop), (XrightOrigin, Ybottom), (XleftOrigin, Ybottom)])) 
            Ytop = Ytop - stepsize
            Ybottom = Ybottom - stepsize
        XleftOrigin = XleftOrigin + stepsize
        XrightOrigin = XrightOrigin + stepsize

    grid = gpd.GeoDataFrame({'geometry':polygons})
    grid.to_file(output_name, driver = "GeoJSON")
    
layout_to_geojson(extent = (-4.511, 4.46, 1.824, 11.54))

In [ ]:
aoi1_polygon = gpd.read_file("../cfg/aois/4.geojson")['geometry'][0]
grid_gdf = gpd.read_file("./supercell_tile_grid.geojson")

aoi1_grid_geometries = grid_gdf.intersection(aoi1_polygon)
aoi1_grid_geometries= aoi1_grid_geometries[aoi1_grid_geometries.is_empty==False]

In [ ]:
aoi1_grid_geometries[aoi1_grid_geometries.geom_type=='LineString'].count()

In [ ]:
aoi1_grid_geometries[aoi1_grid_geometries.geom_type=='Polygon']

In [ ]:
aoi1_grid_geometries.loc[3448740:3448744]

In [ ]:
aoi1_grid_geometries.loc[3251750:3251755][aoi1_grid_geometries.geom_type=='Polygon'].to_file('mapper0_tile_subset.geojson', driver= "GeoJSON")

In [ ]:
test_geoms=aoi1_grid_geometries.loc[3248740:3248742]

gpd.GeoSeries([test_geoms]).to_file('test_tiles.geojson', driver= "GeoJSON")

In [ ]:
gpd.GeoSeries([test_geoms]).to_file('test_tile.geojson', driver= "GeoJSON")

ssh for idlist

In [ ]:
porder idlist --input "/home/rave/cloud-free-planet/notebooks/test_aoi_ghana_larger.geojson" --start "2018-01-01" --end "2018-09-30" --item "PSScene4Band" --asset "analytic_sr" --number 10000 --outfile "/home/rave/cloud-free-planet/notebooks/test_aoi1_id.csv" --cmin 0 --cmax .7 --overlap 100

ssh command for ordering

In [ ]:
porder order --name larger_ghana_aoi --idlist /home/rave/cloud-free-planet/notebooks/test_aoi1_id.txt --item PSScene4Band --asset analytic_sr --boundary /home/rave/cloud-free-planet/notebooks/test_aoi_ghana_larger.geojson --op clip email

In [ ]:
porder multiproc --url https://api.planet.com/compute/ops/orders/v2/d277df43-b783-4b00-ad29-b5df5662af64 --local /home/rave/cloud-free-planet/notebooks/jan-september/

In [ ]:
from subprocess import Popen, PIPE
p = Popen("porder",stdout=PIPE, stderr=PIPE)
stdout, stderr = p.communicate()

In [ ]:
stderr